## Objective

download all genomes in an NCBI accession list table
e.g. [this table](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=SARS-CoV-2,%20taxid:2697049)

In [1]:
# papermill parameters
base="$(homedir())/workspace"
project="sars-cov2-pangenome-analysis"

accession_table_file = "$(base)/$(project)/metadata/sequences.csv"
output_directory = "$(base)/$(project)/data/sequences"

"/home/jovyan/workspace/sars-cov2-pangenome-analysis/data/sequences"

## Materials, Methods, and Functions

In [2]:
mkpath(output_directory)

"/home/jovyan/workspace/sars-cov2-pangenome-analysis/data/sequences"

In [3]:
import Pkg
Pkg.update()

pkgs = [
    "Revise",
    "DataFrames",
    "uCSV",
    "ProgressMeter",
    "FASTX"
]

for pkg in pkgs
    try
        eval(Meta.parse("import $pkg"))
    catch
        Pkg.add(pkg)
        eval(Meta.parse("import $pkg"))
    end
end

# Pkg.add(url="https://github.com/cjprybol/Mycelia.git", rev="master")
import Mycelia

    Updating registry at `/opt/julia/registries/General`
   Installed GraphRecipes ─────── v0.5.12
   Installed DiffResults ──────── v1.1.0
   Installed Plots ────────────── v1.34.4
   Installed HTTP ─────────────── v1.4.0
   Installed Clustering ───────── v0.14.3
   Installed DataFrames ───────── v1.3.6
   Installed Distributions ────── v0.25.75
   Installed Graphs ───────────── v1.7.4
   Installed Glib_jll ─────────── v2.74.0+1
   Installed StaticArraysCore ─── v1.4.0
   Installed DataAPI ──────────── v1.11.0
   Installed RelocatableFolders ─ v1.0.0
   Installed RecipesBase ──────── v1.3.0
   Installed BitFlags ─────────── v0.1.5
   Installed StaticArrays ─────── v1.5.9
   Installed SentinelArrays ───── v1.3.15
   Installed JLFzf ────────────── v0.1.5
   Installed GR_jll ───────────── v0.69.1+0
   Installed Pipe ─────────────── v1.3.0
   Installed fzf_jll ──────────── v0.29.0+0
   Installed FFMPEG_jll ───────── v4.4.2+2
   Installed InverseFunctions ─── v0.1.8
   Installed Tables ───

In [4]:
# @time accession_table = DataFrames.DataFrame(uCSV.read(accession_table_file, typedetectrows=100, header=1, quotes='"')...)

In [5]:
p = ProgressMeter.Progress(countlines(accession_table_file))

ProgressMeter.Progress(6043674, ReentrantLock(nothing, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), 0), 0.1, 0, 1.664564997145336e9, 1.664564997145336e9, 1.664564997145336e9, false, "Progress: ", nothing, ProgressMeter.BarGlyphs('|', '█', ['▏', '▎', '▍', '▌', '▋', '▊', '▉'], ' ', '|'), :green, IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(42) open, 0 bytes waiting))), 0, 0, 0, true, false, 1, 1, Int64[])

In [ ]:
# sort files by reverse size
# this is helpful to track down incomplete sequences
# can also determine the size of each genome
# then determine if there is a good cutoff at which to just remove the incomplete ones
# ls -lShr

# 2_262_407

table_io = open(accession_table_file)
header = readline(table_io)
# ProgressMeter.@showprogress for accession in accession_table[!, "Accession"]
for line in eachline(table_io)
    accession = split(line, ',')[1]
    outfile = joinpath(output_directory, "$(accession).fna")
    if !isfile(outfile) || (filesize(outfile) == 0)
        fasta_records = collect(Mycelia.get_sequence(db="nuccore", accession=accession))
        open(outfile, "w") do io
            fastx_io = FASTX.FASTA.Writer(io)
            for record in fasta_records
                write(fastx_io, record)
            end
            close(fastx_io)
        end
    end
    ProgressMeter.next!(p)
end
table_io

Progress:  64%|██████████████████████████▏              |  ETA: 1:40:48m26mIOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Progress:  64%|██████████████████████████▏              |  ETA: 2:02:29IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Progress:  64%|██████████████████████████▎              |  ETA: 2:59:26IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this li